In [26]:
import cmath
import matplotlib
import sympy as sp
import random as rand
from sympy import init_printing
init_printing()

## Opgave 14

$PR_{n+1}(p)$: Dette er chancen for at du lander på siden p, pagerank.

$(1-d)\frac{1}{N}$: Dette er chancen for at du lander på siden p tilfældigt.

$d \cdot \sum_{q \in inbound(p)}$: Chancen for at lande på q, som linker til p divideret med antallet af sider som q linker til. Dette er ganget med en dæmper


# Opgave 15

In [27]:
W1 = {
    'P1': {'P2', 'P3'},
    'P2': {'P3'},
    'P3': {'P1'},
    'P4': {'P5'},
    'P5': {}
}
W2 = {
    'P1': {'P2'},
    'P2': {'P3'},
    'P3': {'P1'},
    'P4': {'P5'},
    'P5': {'P6'},
    'P6': {'P4'}
}

In [28]:
def counter(p, counts):
    if p in counts:
        counts[p] += 1


In [29]:
chosenp = None
def surf_step(web, p):
    
    # Input: Et netværk som dictionary og en start side
    # Output: Sandsynlighedsfordeling som dictionary for næste hjemmeside
    
    distribution=dict()
    counts = {key: 0 for key in web}
    
    distribution = dict(counts)

    if not p:  
        p = rand.choice(list(web.keys()))
    elif not web[p]: 
        p = rand.choice(list(web.keys()))
    else:
        p = rand.choice(list(web[p]))
    counter(p, counts)

    global chosenp
    chosenp = p

    total = sum(counts.values())
    if total > 0:
        for i in counts:
            distribution[i] = counts[i] / total

    return distribution


In [30]:
import random
def random_surf(web, n):
    
    # Input: Et netværk som dictionary og en start side
    # Output: Sandsynlighedsfordeling som dictionary for næste hjemmeside
    
    ranking = {key: 0 for key in web}

    p = random.choice(list(web.keys()))

    for _ in range(n):
        temp = surf_step(web, p) 
        
        for page in ranking:
            ranking[page] += temp.get(page, 0)

        p =  chosenp

    for page in ranking:
        ranking[page] /= n

    return ranking




In [31]:
import numpy as np

def checkArrayValue(array,value):
    for i in range(0,len(array)):
        if array[i] == value:
            return True
        
    return False

def make_web(n,k,kmin=0):

    # Input: n og k er ikke-negative heltal
    # Output: web er en dictionary med n nøgler.
    # Værdien af hver nøgle er en liste, der er en delmængde af nøglerne.
    
    assert(k < n), "k skal være mindre end n (da man ikke kan linke til sig selv)"
    assert(kmin <= k), "kmin skal være mindre end eller lig med k"
    keys = list(range(n)) # definerer n nøgler fra 0 til n-1 
    web = dict()
    
    for j in keys:
        numlinks = np.random.randint(kmin, k + 1) 
        links = list(np.random.choice([x for x in keys if x != j], size=numlinks, replace=False))
        web[j] = links 

    return web

print(make_web(100,99,50))

{0: [28, 97, 91, 49, 32, 43, 15, 83, 44, 92, 93, 58, 22, 55, 99, 46, 20, 56, 71, 11, 27, 57, 21, 73, 1, 82, 48, 77, 86, 25, 2, 3, 45, 74, 4, 17, 14, 53, 65, 85, 39, 63, 6, 50, 52, 59, 94, 47, 96, 88, 24, 37, 38, 70, 8, 13, 23, 51, 68, 5, 19, 36, 81, 18, 95, 72, 30, 79, 64, 42, 66, 12, 7, 35, 89, 62, 98, 54, 16, 60, 78, 67, 9, 84, 41, 34, 61, 40, 76, 33, 80, 87, 90], 1: [15, 23, 40, 62, 31, 83, 51, 18, 65, 7, 48, 66, 28, 5, 77, 57, 59, 26, 47, 0, 37, 13, 14, 43, 3, 72, 34, 29, 39, 42, 6, 99, 12, 69, 16, 56, 74, 91, 96, 8, 94, 78, 53, 4, 55, 95, 33, 41, 89, 10, 61, 11, 49, 17, 97, 27, 67, 82, 20, 64, 88, 84, 46], 2: [25, 47, 67, 40, 18, 64, 86, 49, 9, 3, 46, 94, 5, 98, 75, 96, 58, 73, 57, 13, 37, 72, 24, 6, 52, 7, 88, 80, 59, 84, 8, 97, 0, 53, 17, 30, 93, 50, 1, 34, 90, 32, 31, 89, 63, 79, 83, 74, 85, 54, 10, 91, 38, 99, 95, 78, 12, 81, 27, 28, 56, 33, 14, 60, 36, 68, 71, 26, 82, 22, 39, 11, 44, 19, 29, 21, 4, 62, 15, 61, 35, 16], 3: [38, 71, 67, 9, 99, 56, 80, 27, 68, 74, 7, 45, 17, 51,

In [32]:
def rank_update(web, PageRanks, page, d):
    new_rank = dict()
    increment = 0.0
    N = len(web) 
    rank_sum = 0.0

    for other_page in web:
        # Check if 'page' is linked to by 'other_page'
        if page in web[other_page]:
            num_links = len(web[other_page])
            if num_links > 0:
                rank_sum += PageRanks[other_page] / num_links
        elif len(web[other_page]) == 0:
            # Handle dangling node: distribute rank evenly
            rank_sum += PageRanks[other_page] / N

    new_rank[page] = (1 - d) / N + d * rank_sum
    
    for p in web:
        if p == page:
            increment = abs(PageRanks[p] - new_rank[p])
        else:
            new_rank[p] = PageRanks[p]
    
    return increment

In [33]:
print(rank_update(W1,random_surf(W1,100),'P4',0.85))

0.030000000000000006


In [34]:
def recursive_PageRank(web, stopvalue=0.0001, max_iterations=200, d=0.85):
    N = len(web)
    start_rank = 1.0 / N
    PageRank_new = {page: start_rank for page in web}
    iteration = 0
    PageRanks = dict()
    
    while iteration < max_iterations:
        iteration += 1
        new_ranks = PageRank_new.copy()
        max_change = 0.0
        
        for page in web:
            change = rank_update(web, PageRank_new, page, d)
            new_ranks[page] += change
            for p in web:
                if p != page:
                    new_ranks[p] -= change/(N-1)
            
        max_change = max(abs(PageRank_new[page] - new_ranks[page]) for page in web)


        PageRanks = new_ranks
        PageRank_new = new_ranks.copy()

        if max_change < stopvalue:
            break
        
    """
    Implementerer den rekursive version af PageRank-algoritmen ved først at oprette
    en PageRank på 1/N til alle sider (hvor N er det samlede antal sider)
    og derefter anvende "rank_update" gentagne gange, indtil en af de to stopbetingelser
    er opnået:
    stopbetingelse 1: den maksimale ændring fra trin n til trin (n+1) over alle PageRank
    er mindre end stopværdien,
    Stopbetingelse 2: antallet af iterationer har nået "max_iterations".

    Input: web er et dictionary som i outputtet af "make_web", d er dæmpningen,
    stopvalue er et positivt float, max_iterations er et positivt heltal.
    """
    return PageRanks, iteration
    

In [35]:
print(recursive_PageRank(W1))


({'P1': -1.6891801084000417e+45, 'P2': -1.1686579858406401e+45, 'P3': -1.8032213721470014e+45, 'P4': 3.480992421598049e+45, 'P5': 1.180067044789635e+45}, 200)
